In [ ]:
import os
import workspace_path
os.chdir(workspace_path.path)
os.getcwd()

## 载入ONNX模型

In [ ]:
onnx_seg = '../mmdeploy/mmseg2onnx_deeplabv3plus'  # 这里载入的是生成的文件夹, end2end.onnx在里面

from mmdeploy_runtime import Segmentor
device = 'cpu'
# device = 'cuda'   # 当使用cuda时, 由于模型推理占用了内核, 所以无法在notebook中执行代码, 需要在终端本地运行
segmentor = Segmentor(onnx_seg, device)

## 颜色配置

In [ ]:
palette = [
    ['background', [127,127,127]],
    ['red', [0,0,200]],
    ['green', [0,200,0]],
    ['white', [144,238,144]],
    ['seed-black', [30,30,30]],
    ['seed-white', [8,189,251]]
]

palette_dict = {}
for idx, each in enumerate(palette):
    palette_dict[idx] = each[1]

## 单帧处理函数

In [ ]:
import time
import cv2
import numpy as np
from mmseg.apis import inference_model

# opacity透明度，越大越接近原图
def process_frame(img_bgr, opacity = 0.3):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    # 语义分割预测
    pred_mask = segmentor(img_bgr)
    # 将预测的整数ID，映射为对应类别的颜色
    pred_mask_bgr = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3))
    for idx in palette_dict.keys():
        pred_mask_bgr[np.where(pred_mask==idx)] = palette_dict[idx]
    pred_mask_bgr = pred_mask_bgr.astype('uint8')
    # 将语义分割预测图和原图叠加显示
    img_bgr = cv2.addWeighted(img_bgr, opacity, pred_mask_bgr, 1-opacity, 0)
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    # 在画面上写字：图片，字符串，左上角坐标，字体，字体大小，颜色，字体粗细
    scaler = 1 # 文字大小
    FPS_string = 'FPS {:.2f}'.format(FPS) # 写在画面上的字符串
    img_bgr = cv2.putText(img_bgr, FPS_string, (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    return img_bgr

## 摄像头拍摄单帧画面

In [ ]:
# 获取摄像头，0为电脑默认摄像头，1为外接摄像头
cap = cv2.VideoCapture(0)
time.sleep(1) # 等一秒，拍照
# 从摄像头捕获一帧画面
success, frame = cap.read()
cap.release() # 关闭摄像头
cv2.destroyAllWindows() # 关闭图像窗口

import matplotlib.pyplot as plt
plt.imshow(frame[:,:,::-1])
plt.show()

## 摄像头实时画面逐帧处理

In [ ]:
cap = cv2.VideoCapture(0)   # 获取摄像头，0为电脑默认摄像头，1为外接摄像头
cap.open(0)     # 打开cap
# 无限循环，直到break被触发
while cap.isOpened():
    
    # 获取画面
    success, frame = cap.read()
    if not success: # 如果获取画面不成功，则退出
        print('获取画面不成功，退出')
        break
    
    ## 逐帧处理
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)
    
    key_pressed = cv2.waitKey(1000) # 每隔多少毫秒毫秒，获取键盘哪个键被按下
    print('按"q"或"esc"键退出, 当前按键为:', key_pressed)

    if key_pressed in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()